    You will find here the Frequently Asked Questions, as well as some other use-case examples that are not part of the User Guide.

1. How to get the top-N recommendations for each user

    Here is an example where we retrieve the top-10 items with highest rating prediction for each user in the MovieLens-100k dataset. We first train an SVD algorithm on the whole dataset, and then predict all the ratings for the pairs (user, item) that are not in the training set. We then retrieve the top-10 prediction for each user.

In [4]:
# From file examples/top_n_recommendations.py
from collections import defaultdict
from surprise import SVD
from surprise import Dataset

In [5]:
def get_top_n(predictions,n = 10):
    '''
    Return the top-N recommendation for each user from a set of predictions.
    
    Args:
        predictions(list of prediction objects): The list of predictions, as 
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
            
    Returns:
        A dict where keys are user (raw) ids and values are lists of tuples:
            [(raw item id, rating estimation), ...] of size n.
    '''
    
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid,iid,true_r,est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

In [6]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [7]:
# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [8]:
top_n = get_top_n(predictions,n = 10)

In [11]:
# Print the recommended items for each user
for uid,user_ratings in top_n.items():
    print(uid,[iid for (iid, _) in user_ratings])

196 ['513', '480', '483', '169', '318', '408', '515', '191', '1194', '511']
186 ['483', '169', '133', '528', '318', '315', '519', '513', '479', '316']
22 ['12', '22', '98', '100', '64', '483', '169', '251', '527', '603']
244 ['134', '286', '187', '59', '474', '136', '223', '14', '60', '611']
166 ['173', '427', '12', '603', '357', '251', '59', '178', '318', '1449']
298 ['251', '169', '64', '313', '216', '173', '408', '12', '1449', '316']
115 ['134', '179', '408', '483', '199', '603', '474', '180', '182', '59']
253 ['174', '963', '176', '172', '169', '114', '528', '408', '134', '191']
305 ['603', '513', '515', '114', '641', '657', '57', '10', '132', '137']
6 ['603', '657', '611', '251', '179', '1007', '428', '654', '190', '641']
62 ['178', '223', '150', '169', '187', '408', '657', '251', '493', '654']
286 ['318', '12', '178', '98', '194', '126', '923', '520', '313', '657']
200 ['181', '427', '64', '12', '194', '316', '408', '190', '657', '114']
210 ['169', '408', '963', '64', '22', '513'

2. How to compute precision@k and recall@k

    Here is an example where we compute Precision@k and Recall@k for each user:

    Precision@k = |{Recommended items that are relevant}| / |{Recommended items}| 
    Recall@k = |{Recommended items that are relevant}| / |{Relevant items}|

    An item is considered relevant if its true rating rui is greater than a given threshold. An item is considered recommended if its estimated rating r^ui is greater than the threshold, and if it is among the k highest estimated ratings.

In [12]:
# From file examples/precision_recall_at_k.py
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold

In [23]:
def precision_recall_at_k(predictions,k=10,threshold=3.5):
    '''
    Return precision and recall at k metrics for each user.
    '''
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid,_, true_r,est,_ in predictions:
        user_est_true[uid].append((est,true_r))
        
    precisions = dict()
    recalls = dict()
    for uid,user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) 
                              for (est,true_r) in user_ratings[:k])
            
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
    
    return precisions, recalls        

In [24]:
data = Dataset.load_builtin('ml-100k')

In [25]:
kf = KFold(n_splits=5)

In [26]:
algo = SVD()

In [29]:
for trainset,testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions,k=5,threshold=4)
    
    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8722870272180984
0.2597001563666203
0.877300070771408
0.261938454179892
0.8813142047467233
0.2601306802415918
0.8774946921443727
0.2754231056084568
0.8666666666666664
0.24988513885866862


3. How to get the k nearest neighbors of a user (or item)

4. How to serialize an algorithm

5. What are raw and inner ids

    Users and items have a raw id and an inner id. Some methods will use/return a raw id (e.g. the predict() method), while some other will use/return an inner id.
    
    Raw ids are ids as defined in a rating file or in a pandas dataframe. They can be strings or numbers. Note though that if the ratings were read from a file which is the standard scenario, they are represented as strings. This is important to know if you’re using e.g. predict() or other methods that accept raw ids as parameters.
    
    On trainset creation, each raw id is mapped to a unique integer called inner id, which is a lot more suitable for Surprise to manipulate. Conversions between raw and inner ids can be done using the to_inner_uid(), to_inner_iid(), to_raw_uid(), and to_raw_iid() methods of the trainset.

6. How to get accuracy measures on the training set

7. How to save some data for unbiased accuracy estimation

8. How to have reproducible experiments

    Some algorithms randomly initialize their parameters (sometimes with numpy), and the cross-validation folds are also randomly generated. If you need to reproduce your experiments multiple times, you just have to set the seed of the RNG at the beginning of your program:

In [3]:
import random
import numpy as np

my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

9. Where are datasets stored and how to change it?

    By default, datasets downloaded by Surprise will be saved in the '~/.surprise_data' directory. This is also where dump files will be stored. You can change the default directory by setting the 'SURPRISE_DATA_FOLDER' environment variable.